In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [2]:
csv_path = os.path.join("..", "..", "data","LINKED_DATA", "TSR_EHR", "TSR_12_CLEANED.csv")
tsr_12 = pd.read_csv(csv_path)
tsr_12.head()

,height_nm,weight_nm,edu_id,pro_id,opc_id,ih_fl,ivtpamg_nm,hospitalised_time,nivtpa_id,nivtpa1_fl,...,nihs_9_out,nihs_10_out,nihs_11_out,total_out,SexName,Age,mrs_tx_1,mrs_tx_3,mrs_tx_6,mrs_tx_12
0,153.0,62.0,3,1,3,0,0.0,8.000,0,999,...,0,1,0,4,0,67.0,1,1,1,1
1,152.0,62.0,3,1,2,0,0.0,4.000,0,999,...,0,0,0,1,0,69.0,1,0,0,0
2,148.0,56.0,2,1,2,0,0.0,5.000,0,999,...,0,0,0,2,0,71.0,0,0,0,0
3,152.0,56.0,4,1,2,0,0.0,3.000,1,0,...,0,0,0,0,0,71.0,0,0,0,0
4,170.0,87.8,5,8,3,0,0.0,8.491,0,999,...,0,0,2,2,1,59.0,5,5,5,4


In [3]:
tsr_12_input = tsr_12.drop(["mrs_tx_12"], axis=1)
tsr_12_input[tsr_12_input == "N"] = 0
tsr_12_input[tsr_12_input == "Y"] = 1
tsr_12_input = tsr_12_input.astype("float64")
tsr_12_input = np.array(tsr_12_input.values)

tsr_12_input_nomrs = tsr_12.drop(["mrs_tx_12", "mrs_tx_6", "mrs_tx_3", "mrs_tx_1"], axis=1)
tsr_12_input_nomrs[tsr_12_input_nomrs == "N"] = 0
tsr_12_input_nomrs[tsr_12_input_nomrs == "Y"] = 1
tsr_12_input_nomrs = tsr_12_input_nomrs.astype("float64")
tsr_12_input_nomrs = np.array(tsr_12_input_nomrs.values)

# 6 classes

In [4]:
tsr_12_output = tsr_12.mrs_tx_1
tsr_12_output = tsr_12_output.astype("float64")
tsr_12_output = np.array(tsr_12_output.values)

## SVM

In [5]:
svr = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores = cross_val_score(svr,tsr_12_input,tsr_12_output,cv = 10, scoring='r2')
print(svr_scores)
print("Mean of R^2:", svr_scores.mean())
print("Std of R^2:", svr_scores.std())

[0.93860922 0.95003529 0.96380571 0.96988535 0.93647074 0.97422336
 0.98135141 0.9911388  0.98521173 0.98100123]
Mean of R^2: 0.9671732846988146
Std of R^2: 0.01847329180352125


In [6]:
acc_svr = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(tsr_12_input, tsr_12_output, test_size=0.3, random_state=i)
    svr.fit(X_train,y_train)
    svr_predict = svr.predict(X_test)
    print(svr_predict)
    svr_predict = np.round(svr_predict)
    print(svr_predict)
    accuracy = (y_test == svr_predict).sum() / len(svr_predict)
    acc_svr.append(accuracy)

print("Accuracy of RF:", acc_svr)
print("Mean of Accuracy of RF:", sum(acc_svr)/10)

[ 6.91053081e-01  8.65993267e-01  3.76339466e+00  6.80580189e-02
  2.11616872e-01  1.04609689e+00  3.12846943e+00  2.08464762e+00
 -6.50718153e-03  9.61412700e-01  1.25395870e+00  1.46651390e+00
  4.99984788e+00  4.03991060e+00  7.94139771e-01  7.30310289e-03
  1.09749440e+00  4.77650345e+00  3.81154824e+00  5.04549918e+00
  2.11076613e-01  1.11346592e+00  4.96135392e+00  8.51519082e-01
  3.12469919e+00  9.72583120e-01  4.47271035e+00  1.09133075e+00
  5.06085346e+00  3.79850703e+00  1.01602727e+00  2.05694300e+00
  2.60515973e+00  4.95855500e+00  7.79139269e-01  9.70934797e-01
  4.78323280e+00  3.77902599e-01  9.88610904e-01  8.35210276e-01
  9.67200660e-01  7.88068829e-01  3.73790040e+00  3.90185697e+00
  1.91868393e+00  4.68667254e-02  9.64453886e-01  4.11857599e+00
  9.05009232e-01  1.01458316e+00  4.50879493e+00  1.04613221e+00
  3.20582613e+00  9.77745768e-01  9.37998883e-02  5.01848385e+00
  2.14342823e+00  1.97193115e+00  1.12919875e+00  4.00315686e+00
  3.05368824e+00  1.01145

[8.32863971e-01 1.05753297e+00 3.14126601e+00 6.68445505e-01
 3.97268870e+00 9.98624532e-01 8.75445537e-01 1.42304400e-01
 2.07112378e+00 3.40561045e-01 4.90805704e+00 1.11484645e+00
 1.64958859e+00 1.86340828e+00 1.04379334e+00 4.54171322e-01
 2.16930626e+00 2.95599057e+00 4.16744658e+00 2.17173928e-01
 1.18391345e+00 4.59473425e+00 2.98114663e+00 1.30846020e+00
 6.15163896e-01 1.15610330e+00 2.37775309e+00 9.95144889e-01
 1.58196362e+00 3.46029994e+00 1.82812655e+00 5.14992836e+00
 1.02123918e+00 2.79681171e+00 4.95333194e+00 4.02287735e+00
 1.00796984e+00 1.96389312e+00 4.00354408e+00 1.53257605e+00
 2.06804616e+00 2.18573257e+00 4.73002672e+00 1.49602472e+00
 1.05507926e+00 1.04648984e+00 2.50602831e+00 2.71766071e+00
 1.68298584e-01 4.02670700e+00 7.61138403e-01 4.04152648e-02
 1.23574140e-01 1.34718251e+00 4.77417169e+00 9.92615316e-01
 1.01136885e+00 1.12712985e+00 1.56986201e+00 3.86913409e+00
 1.09295702e+00 1.15853824e+00 3.13555129e+00 1.95662700e+00
 9.19344020e-01 2.406917

[ 5.  1.  3.  2.  5.  5.  1.  2.  0.  1.  1.  5.  0.  3.  4.  4.  3.  0.
  2.  1.  1.  1.  3.  1.  1.  5.  2.  2.  1.  3.  0.  1.  5.  4.  2.  5.
  2.  1.  2.  0.  3.  4.  5.  3.  4.  1.  4.  1.  4.  0.  0.  1.  4.  3.
  4.  4.  4.  1.  1.  2.  2.  2.  1.  1.  3.  1.  0.  2.  3.  3.  2.  0.
  4.  3.  5.  4.  1.  2.  5.  2.  1.  0.  5.  1.  0.  0.  3.  4.  0.  2.
  2.  1.  5.  2.  4.  1.  1.  1.  3.  5.  1.  1.  2.  1.  3.  2.  3.  1.
  1.  4.  4. -2.  3.  4.  1.  5.  1.  0.  1.  0.  0.  5.  4.  1.  2.  3.
  4.  2.  4.  1.  4.  4.  2.  4.  2.  4.  2.  1.  1.  5.  5.  1.  2.  5.
  5.  3.  4.  4.  1.  4.  4.  3.  3.  3.  1.  1.  3.  0.  4.  4.  4.  1.
  1.  1.  4.  0.  1.  1.  4.  5.  0.  3.  4.  3.  1.  4.  2.  1.  2.  5.
  1.  5.  3.  2.  1.  1.  4.  2.  2.  1.  5.  3.  2.  3.  1.  2.  4.  4.
  0.  4.  4.  5.  1.  1.  1.  1.  1.  3.  3.  2.  5.  4.  0.  4.  5.  1.
  4.  1.  1.  4.  3.  4.  1.  3.  4.  1.  2.  3.  2.  1.  3.  1.  2.  3.
  4.  1.  3.  2.  1.  4.  2.  2.  1.  3.  2.  4.  5

[ 0.  4.  5.  1.  0.  3.  5.  2.  3.  1.  1. -0.  1.  1.  0.  1.  1.  3.
  1.  3.  3.  1.  1.  1.  4.  0.  1.  4.  1.  4.  4.  1.  1.  4.  1.  1.
  1.  0.  4.  0.  3.  4.  4.  1.  0.  5.  5.  2.  4.  0.  3.  5.  0.  2.
  1.  1.  5.  4.  3.  1.  3.  3.  1.  0.  1.  4.  4.  0.  0.  3.  1.  2.
  3.  5.  4.  3.  3.  5.  1.  0.  1.  4.  1.  1.  4.  4.  3.  1.  1.  4.
  3.  3.  5.  4.  1.  1.  5.  4.  2.  4.  2.  1.  0.  1.  0.  4.  4.  5.
  4.  3.  4.  3.  1.  5.  1.  0.  0.  0.  4.  1.  2.  0.  1.  2.  1.  0.
  1.  4.  2.  1.  4.  1.  3.  0.  1.  3.  5.  1.  1.  3.  1.  3.  1.  2.
  1.  3.  0.  3.  1.  4.  4.  2.  1.  3.  4.  2.  2.  2.  1.  0.  0.  2.
  5.  3.  4.  1.  3.  2.  4.  3.  3.  4.  5.  0.  2.  1.  3.  2.  2.  2.
  0.  3.  1.  0.  5.  1.  5.  5.  0.  5.  3.  0.  2.  1.  2.  1.  1.  1.
  4.  1.  3.  2.  4.  1.  5.  1.  2.  3.  0.  1.  5.  4.  0.  4.  3.  4.
  1.  4.  1.  3.  0.  0.  3.  0.  3.  2.  3.  2.  1.  5.  4.  1.  0.  2.
  0.  1.  0.  4.  1.  1.  1.  3.  1.  5.  5.  3.  1

[ 4.  3.  4.  4.  1.  2.  5.  1.  1.  2. -0.  2.  1.  4.  1.  0.  3.  2.
  2.  4.  1.  0.  1.  0.  2.  1.  0.  0.  1.  3.  4.  1.  3.  1.  3.  3.
  2.  4.  3.  4.  4.  3.  0.  3.  1.  1.  5.  2.  2.  3.  2.  3.  1.  0.
  2.  1.  1.  4.  3.  1.  2.  5.  1.  0.  3.  1.  1.  3.  1.  2.  5.  4.
  5.  0.  5.  2.  4.  1.  2.  3.  1.  1.  0.  1.  4.  5.  0.  2.  3.  1.
  5.  0.  1.  5.  4.  3.  4.  4.  4.  1.  1.  4.  4.  5.  5.  3.  4. -0.
  2.  1.  1.  1.  1.  2.  4.  0.  0.  1.  3.  1.  0.  1.  1.  1.  2.  1.
  4.  4.  1.  5.  1.  5.  1.  1.  1.  2.  0.  0.  2.  1.  2.  4.  1.  1.
  0.  1.  3.  1.  5.  1.  1.  1.  3.  1.  2.  4.  5.  3.  2.  5.  1.  1.
  1.  5.  4.  3.  1.  3.  2.  3.  3.  2.  4.  1.  4.  3.  3.  1.  0.  1.
  4.  2.  4.  4.  0.  5.  1.  4.  4.  4.  1.  1.  1.  4.  3.  5.  1.  3.
  2.  0.  3.  3.  4.  0.  2.  4.  5.  3.  0.  5.  1.  1.  1.  5.  2.  3.
  1.  1.  1.  3.  1.  0.  0.  1.  1.  5.  3.  1.  2.  5.  4.  0.  5.  2.
  5.  4.  2.  1.  1.  0.  4.  4.  3.  1.  3.  1.  4

[ 1.76904046e+00  2.02466776e+00  8.91491011e-01  4.69771974e+00
  9.44462375e-02  2.01254285e+00  3.89069505e+00  2.03901765e+00
  2.06470782e-01  5.00183307e+00  4.11428061e+00  1.02450994e+00
 -4.29652207e-02  1.21597146e+00  3.71445876e+00  1.90513232e+00
  2.88378610e+00  1.03527573e+00  2.28451557e-01  5.10739789e+00
  1.99123949e+00  8.41728460e-01  2.05553012e+00  3.49890534e+00
  3.20679015e+00  3.02220432e+00  2.97622786e+00  2.92728402e+00
  8.03742461e-01  4.09712091e+00  4.14047218e+00  1.94372252e+00
  2.15411325e+00  5.72281114e-02  2.03783611e+00  8.21571545e-01
  4.86397010e+00  7.92792543e-01  5.00803406e+00  1.01474998e+00
 -2.51085640e-02  4.13138868e+00  1.02605950e+00  4.26707111e+00
  1.47239223e+00  4.25671044e+00  9.93152179e-01  4.80383412e+00
  1.84331801e+00  8.29862610e-02  4.63518425e+00  1.11035901e+00
  1.98748624e+00  3.29814365e+00  2.01678384e+00  2.12626148e+00
  3.94594586e+00  1.95581503e+00  3.13072296e-01  4.83984506e+00
  1.86760974e+00  2.63028

[4.05036972 4.91876822 0.85897478 2.84325332 0.5774385  0.94334422
 2.36456918 4.07519245 4.94666864 0.82928271 0.12888909 4.14698778
 4.87288355 1.14192914 1.96298014 0.11793282 1.17567546 3.31433779
 1.04978705 4.95319502 3.74462299 1.94694571 2.00562554 1.91562533
 3.97109356 1.11473738 1.45270713 1.01893585 1.04786193 4.31479597
 1.25865183 1.57569642 1.18354261 3.4148645  4.68864463 4.32156505
 0.08631206 1.06539822 5.24414919 1.75826612 4.21837926 1.90242299
 1.56963173 2.10117225 2.87672795 2.59182088 1.04122513 4.72562676
 2.99755756 1.01772443 1.4188929  0.25594673 2.03974084 3.38334948
 3.13954    4.87002386 2.59109494 2.32303127 1.91640266 4.95438
 5.14603986 2.41838807 0.26963598 0.44101931 1.45984985 3.04833212
 1.0791723  5.19438977 1.18991013 0.46822601 1.14805311 1.7279216
 1.41553146 3.21908246 2.00130279 0.1573381  1.60791118 1.03251191
 0.16911524 1.84562805 1.26728175 1.13468824 2.04498459 0.98342093
 0.1099694  4.53016649 2.10844347 0.06635732 3.45117239 0.84187872

[ 1.01972335  4.8386307   1.35284249  0.88866564  2.13803922  3.1406701
  4.16626776  0.09130353  1.48720171  3.44035462  1.67508637  4.50643849
  3.67664066  3.09282456  4.09282643  2.8856927   2.31220045  0.79666766
  2.55422043  4.31376433  1.24213963  0.3509276   2.75051735  1.02402675
  3.73287487  2.79020407  0.72145545  2.15194376  3.098426    4.09291942
  1.92802329  0.93753504  1.10393235  3.10372292  0.99560262  1.03430459
  2.61865636  4.81089939  4.00716326  0.95843731  1.6498831   3.11675968
  3.83615036  3.2825994   0.19497145  0.78763682  1.10492396  0.48611344
  1.9880202   2.02152619  0.20447578  1.97854175  2.07063082  4.40770834
  2.19531035  3.16348991  3.96638998  1.73652179  4.90241408  5.15565726
  1.11329116  1.31118503  2.1199378   4.15800276  4.14213159  1.62486446
  5.12579212  4.09110985  4.77212288  0.67511137  2.91315466  4.16216325
  0.45060835  2.6470569   2.96057537  1.38322825  1.88812688  4.01573984
  4.21385839  0.54419547  4.07610657  4.01969623  3.

[ 1.21609907e-01  1.89987994e+00  3.66917527e+00  9.05242279e-01
  7.70358615e-01  9.11334979e-01  9.96743445e-01  2.87560249e+00
  3.99021463e+00  4.19483365e+00  4.70151197e+00  9.95269972e-01
  4.86149653e+00  2.89539983e-01  2.78150502e+00  2.97745030e+00
  1.12807408e+00  1.35570509e+00  3.00864017e+00  5.20730116e+00
  2.52151497e+00  4.80924125e+00  2.95993923e+00  2.24141938e+00
  4.13993904e+00  3.94955637e+00  4.47006052e+00  3.49879792e+00
  4.27910492e+00  2.35818303e-02  1.04163088e+00  9.28061944e-01
  5.00719007e+00  1.03172617e-01  1.01406559e+00  1.02528136e+00
  1.68261733e+00  4.19581652e+00  3.72525778e+00  3.16439022e+00
  1.04531073e+00  1.03951855e+00  2.01562244e+00  7.25669510e-01
  1.74327011e+00  1.02070803e-02  1.32931166e-01  1.12864383e-01
  2.04560358e+00  8.74561929e-01  5.14623109e+00  2.63378064e+00
  3.81759344e+00  4.17896531e+00  9.95620455e-01  3.44464119e+00
  3.09140809e+00  1.04177490e+00  1.17784918e+00  1.49718798e-01
  3.76700346e+00  3.98155

In [7]:
svr_pred = cross_val_predict(svr,tsr_12_input,tsr_12_output,cv = 10)
svr_pred = np.round(svr_pred)
confusion_matrix(tsr_12_output, svr_pred)

array([[  0,   0,   0,   0,   0,   0,   0],
       [  2, 294,  39,   1,   1,   0,   0],
       [  0,   0, 804,  27,   0,   0,   0],
       [  0,   0,  27, 403,  16,   0,   0],
       [  0,   0,   0,  25, 363,   6,   0],
       [  0,   0,   0,   0,  31, 466,  13],
       [  0,   0,   0,   0,   2,  21, 256]], dtype=int64)

In [8]:
svr_pred[svr_pred == -1] = 0
svr_pred[svr_pred == -2] = 0
svr_pred[svr_pred == 6] = 5
confusion_matrix(tsr_12_output, svr_pred)

array([[296,  39,   1,   1,   0,   0],
       [  0, 804,  27,   0,   0,   0],
       [  0,  27, 403,  16,   0,   0],
       [  0,   0,  25, 363,   6,   0],
       [  0,   0,   0,  31, 466,  13],
       [  0,   0,   0,   2,  21, 256]], dtype=int64)

In [9]:
svr1 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores1 = cross_val_score(svr1,tsr_12_input_nomrs,tsr_12_output,cv = 10, scoring='r2')
print(svr_scores1)
print("Mean of R^2:", svr_scores1.mean())
print("Std of R^2:", svr_scores1.std())

[0.70748733 0.77261478 0.83822475 0.80167537 0.77989062 0.8290307
 0.79898991 0.82682862 0.71824708 0.66624142]
Mean of R^2: 0.7739230595830747
Std of R^2: 0.05524063162541846


In [10]:
acc_svr1 = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(tsr_12_input_nomrs, tsr_12_output, test_size=0.3, random_state=i)
    svr1.fit(X_train,y_train)
    svr1_predict = svr1.predict(X_test)
    print(svr1_predict)
    svr1_predict = np.round(svr1_predict)
    print(svr1_predict)
    accuracy = (y_test == svr1_predict).sum() / len(svr1_predict)
    acc_svr1.append(accuracy)

print("Accuracy of RF:", acc_svr1)
print("Mean of Accuracy of RF:", sum(acc_svr1)/10)

[ 0.56567464  0.59945181  3.83966144  0.74863464  0.55365784  1.03543937
  2.96311738  2.10466049  0.57275407  1.18306262  1.28450092  1.24867498
  4.27075192  3.86289123  1.43928353  0.45059531  0.79287619  4.47050755
  3.43755731  4.78968055  1.03893548  0.95435503  4.61315875  0.96724954
  1.64436108  0.92641206  2.92966102  1.48421603  4.9139679   3.46244157
  0.96865191  2.16521075  2.50423178  4.85630378  1.23534786  0.89390263
  3.65804248  1.57272099  1.0790187   3.11554472  0.77489204  0.88424485
  2.86771688  3.44958948  2.61395654  0.57358025  1.01675645  4.2062941
  0.7300339   1.04948204  4.55688173  1.07279533  3.17097315  1.17161534
  0.59155757  4.75966333  1.04643498  1.20176882  1.11384697  3.30829946
  2.67409093  1.03053284  0.85728218  2.86648684  3.45934955  1.71446666
  0.79506818  2.34266102  1.33033686  1.32869266  0.89348472  0.52460845
  4.12467659  0.82100224  2.09836292  2.08121435  1.12301441  2.94313209
  4.57453079  2.54245316  2.20001552  0.971252    1.

[0.5995547  0.8413782  4.84046529 0.91831353 4.06865956 0.7489353
 0.89656901 0.62332753 2.49407653 1.05379572 5.2011097  1.07446445
 2.82923482 1.55872076 1.01961099 0.59990839 2.07269384 2.6610119
 3.27202156 0.75657604 1.35387943 4.43213334 2.60503797 0.91681057
 1.29166932 1.31587524 2.87004396 0.89257974 0.92665674 2.97145142
 1.56471835 4.72646785 1.03931551 2.72034678 4.95208039 4.04337663
 0.89359243 2.73049549 3.07416174 1.29112214 1.9264777  2.21782357
 4.4303614  0.98152346 0.98391783 1.75971508 3.66146132 3.66922379
 0.70106062 3.45685982 0.61689064 0.28767592 0.87497935 1.72684586
 4.54965421 0.93196733 1.03633235 1.31148358 2.76579383 3.17212801
 1.11093817 1.50407957 3.36360228 1.24385762 1.70783088 2.7357081
 0.98928641 4.01878234 1.18842956 0.48090617 5.35148024 1.02692259
 1.11909257 0.62315107 4.82217492 0.71063608 3.85945522 1.37239354
 1.27553147 3.78506278 0.47682121 0.57754753 1.00142503 1.14128341
 2.72018299 1.47789613 2.32135123 2.65991505 3.26142735 4.7167971

[ 4.42566087  0.67317509  2.37336444  2.80697843  4.8580902   5.24212169
  1.008433    1.48394477  0.43226905  0.97955181  1.30881057  4.6992682
  0.69329933  3.08760358  4.37328096  3.28825267  3.53012793  0.57770024
  1.31316154  1.10485979  1.55165718  1.24398908  2.93249074  0.99360251
  1.58738602  5.15737743  2.28911136  1.97873816  1.11002899  2.47840701
  0.55860197  0.82954788  4.45532178  3.69466418  2.303071    4.34790319
  2.28279306  1.18813949  3.76691128  0.68875811  3.10081091  2.75152647
  4.83617138  2.03168064  3.25573387  1.78797408  3.36147302  1.06959554
  2.17291693  0.76625739  0.51216695  1.24277925  3.34130664  3.21798679
  4.33606656  4.45317395  4.97312465  1.01823136  0.85188708  0.9804877
  2.46597322  1.55514017  0.96088353  1.25107203  2.42280034  1.04885226
  0.75799845  3.04838191  5.00963976  2.6583168   1.42697056  0.85835442
  4.27808955  1.93008191  4.86277254  3.69844518  0.78049757  2.89055078
  4.64165303  2.17576651  0.93399127  0.62669599  5.4

[ 1.01426716  1.81783196  4.99360686  1.23063538  1.02112604  3.26044354
  4.07379726  1.14423685  2.93642581  3.22309938  1.14051962  0.20493753
  0.9396692   0.79094938  0.95284654  0.70387467  2.22771528  2.1889824
  0.76619331  3.18240702  3.12909934  1.15452151  0.91539128  1.14144323
  2.52808701  0.41497258  0.78669717  4.49308134  1.02339244  4.34930595
  3.48995188  0.67092426  1.17756035  4.48406326  1.38529804  0.74517084
  0.89403937  0.59963207  3.84947467  0.59325628  2.03577128  4.6446704
  3.3664567   1.17029583  0.77434443  5.18625167  4.72011794  1.60314205
  2.86424937  0.73044356  3.04804816  4.61313597  0.63999758  1.8713093
  0.89072     0.31856363  5.02630456  3.03783748  3.36243487  0.56120861
  2.95691848  5.42247032  0.82655286  0.26109181  1.89962648  3.29808781
  4.29685093  0.67243032  0.29974405  3.67545243  0.96392425  1.2987784
  3.33549284  3.38650981  4.02029197  2.6804761   4.05324043  3.86242516
  0.85779157  0.34450543  1.09089536  4.55560705  0.639

[ 4.32455751  2.82202851  3.71201798  3.68634605  2.50103352  2.23836196
  5.15777442  1.27538085  1.55593647  2.52328952  0.42154924  2.79999821
  0.32925179  4.12178434  0.78835277  0.57436214  2.12240237  2.56002608
  1.77623579  3.60330081  0.97462882  0.72109992  1.05620235  0.65812623
  1.14563158  0.73470206  0.91478929  0.50494804  1.8933948   3.03998633
  4.4615849   1.61164441  2.2959716   0.49756677  2.36522441  2.95303922
  0.9551589   3.98264313  2.52833038  4.07168279  3.37888082  3.25864642
  0.71817739  2.85667224  1.12925686  0.8525956   4.6985094   1.3095511
  1.87369752  3.23093512  1.48803251  2.35887352  0.9181644   0.80158917
  1.56568456  1.16432384  1.0895419   4.10023007  2.12852183  1.02856555
  1.31338973  4.5937491   2.63300011  0.92635478  2.83025996  1.53436923
  0.3889975   2.06271635  1.67918709  2.6693069   4.73001253  3.77862577
  4.90931739  0.53125588  4.45286755  2.91417377  3.89070333  0.8177916
  1.87052859  2.51235214  1.68487242  1.12184425  0.4

[ 1.46662501  1.42243979  0.99508387  4.38532925  0.47840087  2.19069765
  3.09118914  2.09880388  0.51177345  5.50334038  3.7548959   0.96424072
  0.54946923  1.73611119  2.85662799  2.76435689  2.6216371   1.1094813
  0.94472808  5.0551069   1.74327715  1.29753901  1.80086939  3.38818985
  3.9347812   3.68332247  2.66203738  2.16111473  0.8775607   3.42595494
  4.35516864  1.01621878  1.99119887  0.48640181  2.1754006   0.57202677
  4.40540676  0.98519984  4.84603652  1.09316043  0.81900313  4.31196318
  0.74970524  4.34015887  1.73418249  4.91950462  1.00443413  4.70283879
  1.85522576  0.44256031  3.80030731  1.43095624  1.90567152  3.42090517
  1.79719177  3.32915219  3.00116206  2.24932907  0.76267153  4.86775695
  2.13417894  2.68193162  4.10499485  2.54878979  5.21509067  0.99624503
  1.30251867  3.54711697  1.21866046  3.48443158  0.83836063  1.34134963
  0.68269582  2.9565344   1.25870297  0.61749373  4.78963174  3.27973299
  4.24771673  1.26662927  0.47806864  0.95472252  2.

[ 0.84273575  4.50071351  2.46461085  1.10136894  0.86422649  3.81299523
  4.21444796  0.51188155  3.30412677  2.19045114  1.80440311  4.18905927
  2.7922891   2.88427081  4.0653175   2.64268124  2.45966652  0.73434497
  2.74469318  4.58782196  1.18919528  0.66231713  3.03430241  1.10790572
  3.42506329  3.40122643  0.9133962   2.07731937  3.10843665  3.05255618
  1.80347482  0.94013703  1.09410343  2.51616222  0.78938691  1.10185214
  2.49751894  4.79930225  4.42343383  0.98985879  0.92872039  2.6875799
  2.1442038   3.11145261  1.51905236  0.83700339  1.92586904  1.84958516
  2.29877469  3.11481985  1.28362824  2.22666551  2.15319242  1.87475192
  1.25868379  3.21531321  4.54230904  0.91314199  4.22843135  5.25009474
  1.20631257  2.28998211  2.37119498  4.2304956   4.34392113  1.15581165
  4.75879325  3.39218056  3.59809154  0.99585482  2.74589393  4.59405403
  0.62668376  2.35409379  2.90403708  2.84203464  1.68101611  3.78869281
  4.60897945  0.53695445  4.61618273  3.79153148  3.

In [11]:
svr_pred1 = cross_val_predict(svr1,tsr_12_input_nomrs,tsr_12_output,cv = 10)
svr_pred1 = np.round(svr_pred1)
confusion_matrix(tsr_12_output, svr_pred1)

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  1,   1,  82, 215,  30,   7,   0,   1,   0],
       [  0,   0,  14, 658, 125,  28,   6,   0,   0],
       [  0,   0,   1, 169, 196,  73,   7,   0,   0],
       [  0,   0,   0,  17, 101, 235,  34,   7,   0],
       [  0,   0,   0,   4,  18, 156, 277,  55,   0],
       [  0,   0,   0,   1,   5,  15,  80, 173,   5],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=int64)

In [12]:
svr_pred1[svr_pred1 == -5] = 0
svr_pred1[svr_pred1 == -4] = 0
confusion_matrix(tsr_12_output, svr_pred1)

array([[  0,   0,   0,   0,   0,   0,   0,   0],
       [  1,  83, 215,  30,   7,   0,   1,   0],
       [  0,  14, 658, 125,  28,   6,   0,   0],
       [  0,   1, 169, 196,  73,   7,   0,   0],
       [  0,   0,  17, 101, 235,  34,   7,   0],
       [  0,   0,   4,  18, 156, 277,  55,   0],
       [  0,   0,   1,   5,  15,  80, 173,   5],
       [  0,   0,   0,   0,   0,   0,   0,   0]], dtype=int64)

## RF

In [13]:
rfr = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores = cross_val_score(rfr,tsr_12_input,tsr_12_output,cv = 10, scoring='r2')
print(rfr_scores)
print("Mean of R^2:", rfr_scores.mean())
print("Std of R^2:", rfr_scores.std())

[0.99995718 0.99966588 0.9999666  0.99991394 0.99951801 0.9999538
 0.99990526 0.99996546 0.99998622 0.99988793]
Mean of R^2: 0.999872028686078
Std of R^2: 0.0001468289738189413


In [14]:
acc_rfr = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(tsr_12_input, tsr_12_output, test_size=0.3, random_state=i)
    rfr.fit(X_train,y_train)
    rfr_predict = rfr.predict(X_test)
    print(rfr_predict)
    rfr_predict = np.round(rfr_predict)
    print(rfr_predict)
    accuracy = (y_test == rfr_predict).sum() / len(rfr_predict)
    acc_rfr.append(accuracy)

print("Accuracy of RF:", acc_rfr)
print("Mean of Accuracy of RF:", sum(acc_rfr)/10)

[1.         1.         4.         0.         0.         1.
 3.         2.         0.         1.         1.         1.
 5.         4.         1.         0.         1.         5.
 4.         5.         0.         1.         5.         1.
 3.         1.         5.         1.         5.         4.
 1.         2.         3.         5.         1.         1.
 5.         0.         1.         1.         1.         1.
 4.         4.         2.         0.         1.         4.
 1.         1.         5.         1.         3.         1.
 0.         5.         2.         2.         1.         4.
 3.         1.         0.         2.         1.         1.
 0.         3.         1.         1.         1.         0.
 4.         1.         3.         4.         1.         3.
 5.         3.         3.         1.         1.         1.
 1.         4.06666667 5.         1.         5.         0.
 1.         3.         5.         3.         4.         1.
 4.         1.         4.         3.         5.         

[1.         1.         3.         1.         4.         1.
 1.         0.         2.         0.         5.         1.
 1.         2.         1.         0.         2.         3.
 4.         0.         1.         5.         3.         1.
 0.         1.         2.         1.         2.         4.
 2.         5.         1.         3.         5.         4.
 1.         1.         4.         2.         2.06666667 2.
 5.         2.         1.         1.         3.         2.
 0.         4.         1.         0.         0.         1.
 5.         1.         1.         1.         1.         4.
 1.         1.         3.         2.         1.         3.
 2.         4.06666667 1.         0.         5.         2.
 1.         0.         5.         0.         4.         2.
 2.         4.         1.         0.         0.         2.
 4.         1.         0.         3.         2.06666667 5.
 4.         3.         1.         2.         2.         0.
 2.         1.         0.         2.         3.         

[4.         1.         3.         3.         5.         5.
 1.         2.         0.         1.         1.         5.
 0.         3.         4.         4.         3.         0.
 2.         1.         1.         1.         3.         1.
 1.         5.         2.         2.         1.         3.
 0.         1.         5.         4.         3.06666667 4.
 1.         1.         1.         0.         3.         4.
 5.         3.         4.         1.         4.         1.
 4.         0.         0.         2.         4.         3.
 4.         4.         4.         1.         1.         2.
 2.         2.         1.         1.         3.         1.
 1.         2.         0.26666667 3.         3.         0.
 4.         2.         5.         4.         0.06666667 1.
 5.         2.         1.         0.         5.         1.
 0.         0.         3.         4.         0.         2.
 2.         1.         5.         2.         4.         1.
 1.         1.         3.         5.         1.         

[0.06666667 4.93333333 5.         1.13333333 0.         3.
 5.         2.         3.         0.06666667 1.         0.
 1.         1.         0.         1.         1.06666667 3.
 1.         3.         3.         1.         1.         1.
 4.         0.         1.         4.06666667 1.         4.
 4.         0.13333333 1.         4.         1.         1.
 1.         0.         4.         0.         3.         4.
 4.         1.         0.         5.         5.         2.
 4.         0.         3.         5.         0.         2.
 1.         0.         5.         4.         4.         1.
 3.         1.         1.         0.         1.         4.
 4.         0.06666667 0.         2.8        1.         2.
 3.         5.         4.         3.         3.         5.
 1.         0.         1.         4.         1.         1.
 4.         4.         3.         0.         1.         4.
 4.         3.         5.         4.         1.         1.
 5.         4.         2.         5.         2.         

[3.         3.         4.         4.         0.         2.
 5.         1.         1.         2.         0.         2.
 0.         4.         1.         0.         3.         2.
 1.         4.         1.         0.         1.         0.
 2.         1.         0.         0.         1.         3.
 4.         1.         3.         1.         3.         3.
 3.         4.         3.         4.         4.         3.
 0.         3.         1.         1.         5.         3.
 1.         3.         2.         3.         1.         0.
 2.         2.         1.         4.         3.         1.
 2.         5.         1.         0.         3.         1.
 1.         3.         2.         1.         5.         4.
 5.         0.         5.         2.         5.         1.
 3.         4.         0.26666667 1.         0.         1.
 4.         5.         0.         2.         3.         0.13333333
 5.         0.         1.         5.         4.         4.
 4.         4.         4.         0.         0. 

[3. 3. 4. 4. 0. 2. 5. 1. 1. 2. 0. 2. 0. 4. 1. 0. 3. 2. 1. 4. 1. 0. 1. 0.
 2. 1. 0. 0. 1. 3. 4. 1. 3. 1. 3. 3. 3. 4. 3. 4. 4. 3. 0. 3. 1. 1. 5. 3.
 1. 3. 2. 3. 1. 0. 2. 2. 1. 4. 3. 1. 2. 5. 1. 0. 3. 1. 1. 3. 2. 1. 5. 4.
 5. 0. 5. 2. 5. 1. 3. 4. 0. 1. 0. 1. 4. 5. 0. 2. 3. 0. 5. 0. 1. 5. 4. 4.
 4. 4. 4. 0. 0. 4. 4. 5. 5. 3. 4. 0. 2. 1. 1. 1. 0. 2. 4. 0. 0. 1. 3. 1.
 0. 1. 1. 1. 2. 1. 4. 4. 1. 5. 1. 5. 1. 1. 1. 2. 0. 0. 2. 1. 2. 4. 0. 1.
 0. 1. 3. 1. 5. 1. 1. 2. 3. 1. 1. 4. 5. 3. 2. 5. 1. 1. 1. 5. 4. 3. 1. 3.
 1. 3. 3. 2. 4. 1. 4. 3. 4. 1. 0. 1. 4. 3. 4. 4. 0. 5. 1. 4. 4. 4. 1. 1.
 1. 4. 3. 5. 1. 3. 2. 0. 3. 3. 4. 0. 2. 4. 5. 3. 0. 5. 1. 1. 1. 5. 2. 3.
 1. 1. 1. 4. 1. 0. 0. 1. 1. 5. 4. 1. 2. 5. 4. 0. 5. 2. 5. 4. 2. 1. 0. 0.
 5. 4. 3. 1. 3. 1. 4. 2. 0. 4. 1. 0. 0. 1. 4. 1. 4. 4. 2. 4. 1. 0. 4. 1.
 1. 4. 1. 0. 4. 4. 5. 4. 1. 5. 4. 1. 3. 3. 1. 1. 1. 1. 1. 4. 4. 1. 1. 0.
 2. 1. 1. 5. 3. 1. 5. 1. 2. 3. 0. 4. 0. 3. 4. 5. 0. 4. 3. 1. 5. 4. 1. 2.
 1. 1. 0. 4. 1. 1. 1. 1. 5. 0. 1. 0. 4. 3. 5. 4. 0.

[2.         2.         1.         5.         0.         2.
 4.         2.         0.         5.         4.         1.
 0.         1.         4.         2.         3.         1.
 0.         5.         2.         1.         2.         4.
 3.         3.         3.         3.         1.         4.
 4.         2.         2.         0.         2.         1.
 5.         1.         5.         1.         0.         4.
 1.         4.         1.         4.         1.         5.
 2.         0.         5.         1.         2.         3.
 2.         2.         4.         2.         0.         5.
 2.         3.         5.         3.         5.         1.
 1.         4.         3.         4.         1.         2.
 1.         3.         0.         1.         5.         3.
 5.         3.06666667 0.         2.         2.         2.
 1.         3.         1.         1.         0.         4.
 4.         1.         4.         4.         4.         1.
 4.         1.         4.         0.         1.         

[1.         5.         1.         1.         3.         3.
 4.         0.         1.         4.         2.         4.93333333
 4.         3.         4.         3.         2.         1.
 2.         4.86666667 1.         0.         3.         1.
 4.         3.         1.         2.         2.86666667 4.
 2.         1.         1.         3.         1.         1.
 3.         5.         4.         1.         2.         3.
 4.         3.         0.         1.         1.         0.
 2.         2.         0.         2.         2.         5.
 2.         3.         4.         2.         5.         5.
 1.         1.         2.         4.         4.         2.
 5.         4.         5.         1.         3.         4.
 0.         3.         3.         1.         2.         4.
 4.         1.         4.         4.         3.         2.
 1.         2.         2.         0.         0.         4.
 0.         2.         3.         1.         1.         0.
 2.         0.         4.         4.         0. 

In [15]:
rfr_pred = cross_val_predict(rfr,tsr_12_input,tsr_12_output,cv = 10)
rfr_pred = np.round(rfr_pred)
confusion_matrix(tsr_12_output, rfr_pred)

array([[337,   0,   0,   0,   0,   0],
       [  0, 831,   0,   0,   0,   0],
       [  0,   0, 446,   0,   0,   0],
       [  0,   0,   0, 394,   0,   0],
       [  0,   0,   0,   0, 510,   0],
       [  0,   0,   0,   0,   0, 279]], dtype=int64)

In [16]:
rfr1 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores1 = cross_val_score(rfr1,tsr_12_input_nomrs,tsr_12_output,cv = 10, scoring='r2')
print(rfr_scores1)
print("Mean of R^2:", rfr_scores1.mean())
print("Std of R^2:", rfr_scores1.std())

[0.7847378  0.81460552 0.86070665 0.81994466 0.78999239 0.83301935
 0.83633177 0.86456348 0.77757829 0.78390643]
Mean of R^2: 0.8165386343479961
Std of R^2: 0.030388623691578846


In [17]:
acc_rfr1 = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(tsr_12_input_nomrs, tsr_12_output, test_size=0.3, random_state=i)
    rfr1.fit(X_train,y_train)
    rfr1_predict = rfr1.predict(X_test)
    print(rfr1_predict)
    rfr1_predict = np.round(rfr1_predict)
    print(rfr1_predict)
    accuracy = (y_test == rfr1_predict).sum() / len(rfr1_predict)
    acc_rfr1.append(accuracy)

print("Accuracy of RF:", acc_rfr1)
print("Mean of Accuracy of RF:", sum(acc_rfr1)/10)

[0.06666667 0.2        3.73333333 0.13333333 0.06666667 1.
 2.73333333 2.73333333 0.26666667 1.         1.6        1.2
 4.93333333 3.93333333 1.6        0.13333333 1.46666667 4.73333333
 4.13333333 4.86666667 1.         1.53333333 4.26666667 1.
 2.66666667 1.06666667 3.13333333 1.         4.73333333 3.33333333
 0.6        1.33333333 2.66666667 5.         1.06666667 1.
 4.06666667 0.33333333 1.2        4.         1.2        1.
 3.33333333 3.46666667 2.53333333 0.06666667 1.         3.86666667
 1.         1.2        4.26666667 1.         3.73333333 1.2
 0.06666667 4.4        1.4        1.73333333 1.6        2.93333333
 2.4        0.86666667 1.06666667 2.73333333 2.8        0.6
 0.06666667 2.66666667 1.6        1.4        1.         0.26666667
 3.86666667 1.         3.53333333 1.8        1.06666667 2.86666667
 4.8        1.93333333 2.8        0.86666667 1.06666667 0.8
 0.93333333 2.2        4.26666667 1.4        5.         0.8
 0.86666667 2.6        4.66666667 4.2        3.6        1.2
 3

[0.93333333 0.8        4.93333333 1.46666667 4.13333333 1.
 1.06666667 0.06666667 2.06666667 1.13333333 4.93333333 1.
 2.53333333 2.73333333 1.         0.53333333 1.66666667 3.
 3.6        0.         2.06666667 4.33333333 3.8        1.06666667
 0.93333333 1.26666667 3.2        0.93333333 1.4        3.8
 1.6        4.66666667 1.         3.06666667 4.2        3.93333333
 0.73333333 1.         3.73333333 1.73333333 1.33333333 2.53333333
 4.4        1.8        1.13333333 1.33333333 4.         3.46666667
 0.06666667 3.86666667 1.46666667 0.2        0.13333333 1.26666667
 3.93333333 1.         0.93333333 0.8        2.4        3.46666667
 0.86666667 1.86666667 2.53333333 1.73333333 1.53333333 2.93333333
 1.46666667 4.2        1.         0.2        5.         0.93333333
 1.         0.06666667 4.86666667 0.46666667 4.         1.66666667
 1.66666667 3.53333333 0.2        0.26666667 1.         1.66666667
 3.8        0.8        2.13333333 2.13333333 3.73333333 4.46666667
 4.2        2.6        0.8

[4.66666667 0.73333333 1.73333333 3.06666667 4.66666667 4.53333333
 1.06666667 1.53333333 0.06666667 1.         1.73333333 4.86666667
 0.4        2.86666667 4.06666667 4.13333333 3.8        0.46666667
 1.8        1.26666667 0.93333333 1.2        2.86666667 1.66666667
 1.53333333 5.         3.         2.         0.86666667 2.93333333
 1.46666667 0.4        4.46666667 4.         2.06666667 4.
 2.26666667 1.66666667 3.6        0.86666667 3.26666667 2.8
 4.8        2.53333333 3.6        2.33333333 4.33333333 1.
 1.86666667 0.2        0.33333333 1.4        3.13333333 3.2
 4.06666667 3.93333333 4.8        1.         1.13333333 1.53333333
 3.         1.8        0.86666667 1.8        2.73333333 1.53333333
 0.         2.93333333 4.93333333 2.46666667 1.33333333 0.8
 4.         1.53333333 4.8        3.93333333 0.13333333 2.66666667
 5.         2.2        0.86666667 0.26666667 4.6        0.86666667
 0.33333333 0.13333333 2.73333333 4.33333333 0.93333333 1.53333333
 1.66666667 1.46666667 4.7333333

[0.8        1.         4.13333333 2.         1.         3.06666667
 4.66666667 1.53333333 2.86666667 3.33333333 1.13333333 0.13333333
 0.73333333 1.06666667 0.4        1.         2.53333333 1.73333333
 1.         2.53333333 3.13333333 1.8        0.93333333 1.53333333
 2.93333333 0.06666667 0.73333333 4.6        1.         4.33333333
 3.86666667 0.06666667 0.86666667 3.93333333 1.8        1.
 0.93333333 0.13333333 3.6        1.2        2.66666667 3.86666667
 3.86666667 1.         0.26666667 4.66666667 4.26666667 2.
 2.8        0.13333333 3.06666667 4.4        0.2        1.86666667
 1.26666667 0.33333333 4.8        3.33333333 3.93333333 0.93333333
 3.13333333 0.93333333 1.         0.2        1.33333333 3.33333333
 4.13333333 0.4        0.         3.93333333 1.06666667 1.6
 2.86666667 3.6        3.86666667 2.86666667 3.93333333 4.6
 1.         0.13333333 1.06666667 4.13333333 1.         1.
 3.33333333 4.13333333 2.86666667 1.         1.8        3.93333333
 2.         3.13333333 4.6       

[3.46666667 2.93333333 4.26666667 4.06666667 2.53333333 2.46666667
 4.93333333 1.66666667 0.8        2.2        0.         2.53333333
 0.         3.86666667 0.         0.         1.46666667 2.53333333
 1.26666667 4.2        0.8        0.2        1.         0.2
 1.46666667 1.         0.         0.06666667 1.33333333 2.86666667
 4.33333333 1.8        2.8        0.8        2.6        3.
 1.06666667 4.13333333 3.06666667 4.33333333 3.06666667 2.93333333
 0.8        2.86666667 1.13333333 0.8        4.66666667 0.93333333
 1.73333333 2.6        1.6        3.         0.93333333 0.93333333
 1.86666667 1.         1.46666667 3.8        2.6        0.86666667
 1.4        4.73333333 2.66666667 0.53333333 2.86666667 1.8
 0.         1.06666667 1.6        3.         5.         3.93333333
 4.86666667 0.         5.         2.93333333 4.46666667 0.93333333
 1.73333333 2.6        1.8        1.06666667 0.06666667 1.06666667
 4.         5.         0.66666667 1.66666667 2.93333333 0.
 3.73333333 0.06666667 0.

[1.6        0.93333333 0.86666667 2.13333333 1.6        3.86666667
 0.8        2.         3.4        4.         0.93333333 3.93333333
 2.13333333 1.53333333 4.2        2.2        2.2        1.06666667
 1.6        2.8        2.6        0.86666667 1.33333333 4.93333333
 0.86666667 3.13333333 4.         0.46666667 2.6        4.13333333
 1.         1.6        4.8        4.53333333 4.53333333 4.6
 4.26666667 3.4        3.86666667 0.93333333 3.13333333 1.6
 1.66666667 0.4        2.8        3.93333333 4.53333333 0.33333333
 0.86666667 1.33333333 1.33333333 0.13333333 1.93333333 1.86666667
 2.46666667 0.93333333 1.         0.06666667 2.73333333 1.93333333
 3.6        1.73333333 4.06666667 0.86666667 0.66666667 0.86666667
 1.6        3.4        0.93333333 0.06666667 1.6        4.33333333
 4.26666667 1.66666667 2.2        0.93333333 3.66666667 0.13333333
 1.73333333 1.26666667 3.93333333 2.86666667 1.06666667 4.
 2.8        5.         4.66666667 1.46666667 1.8        1.6
 1.33333333 2.         2

[1.73333333 1.46666667 1.53333333 4.66666667 0.         1.46666667
 2.93333333 1.73333333 0.13333333 4.73333333 3.8        1.46666667
 0.2        2.         3.33333333 2.33333333 2.6        0.93333333
 0.86666667 4.66666667 2.33333333 1.46666667 1.6        3.86666667
 3.53333333 3.73333333 2.26666667 2.66666667 1.2        3.66666667
 4.2        1.8        1.66666667 0.         2.4        0.93333333
 4.06666667 0.6        4.86666667 1.2        0.4        4.26666667
 0.73333333 3.46666667 1.86666667 4.86666667 0.93333333 4.86666667
 1.73333333 0.06666667 3.8        1.06666667 1.53333333 3.2
 1.73333333 2.86666667 3.6        2.46666667 1.         4.26666667
 2.8        2.6        4.53333333 2.06666667 5.         1.
 1.53333333 3.26666667 1.4        3.73333333 1.06666667 1.73333333
 0.93333333 2.8        1.06666667 1.06666667 4.53333333 3.8
 4.6        1.8        0.         0.73333333 2.66666667 4.06666667
 0.86666667 3.2        1.6        1.73333333 1.06666667 3.4
 4.8        1.26666667 3

[4.26666667 4.8        1.         2.53333333 0.33333333 0.86666667
 2.86666667 3.73333333 4.8        1.46666667 0.13333333 4.06666667
 4.93333333 0.8        1.66666667 0.06666667 1.6        3.93333333
 0.93333333 4.13333333 3.86666667 1.66666667 1.86666667 1.46666667
 3.8        0.86666667 2.2        1.         0.93333333 4.13333333
 1.06666667 2.73333333 2.         3.2        4.33333333 3.86666667
 0.4        1.66666667 4.8        1.46666667 4.33333333 1.6
 1.26666667 1.6        3.93333333 2.6        0.8        4.4
 2.66666667 1.66666667 1.8        0.4        2.8        2.93333333
 3.06666667 4.86666667 2.86666667 2.6        1.6        4.86666667
 4.86666667 2.8        0.2        0.73333333 1.6        2.86666667
 1.53333333 5.         2.13333333 0.13333333 1.13333333 2.86666667
 0.93333333 3.06666667 2.86666667 0.         0.93333333 1.2
 0.33333333 2.         0.86666667 2.06666667 1.86666667 1.53333333
 0.46666667 4.8        1.53333333 0.53333333 3.         0.86666667
 3.86666667 3.86

[1.         4.53333333 2.33333333 0.93333333 1.06666667 3.73333333
 4.26666667 0.86666667 3.13333333 2.53333333 1.73333333 4.33333333
 3.73333333 2.8        4.13333333 2.8        2.         1.
 2.93333333 4.66666667 1.6        1.13333333 3.53333333 1.
 3.66666667 2.26666667 0.93333333 1.73333333 3.2        3.53333333
 1.73333333 1.         0.93333333 2.53333333 1.         1.
 3.13333333 4.6        4.4        0.93333333 1.66666667 2.2
 2.86666667 3.73333333 0.86666667 1.13333333 1.73333333 2.8
 2.06666667 3.93333333 1.2        1.8        2.8        2.46666667
 2.53333333 3.26666667 4.2        1.4        4.06666667 4.86666667
 1.46666667 2.06666667 2.86666667 4.13333333 4.06666667 1.6
 4.8        3.93333333 3.86666667 0.73333333 2.46666667 4.13333333
 1.13333333 3.         2.8        3.2        1.33333333 3.93333333
 4.         0.26666667 4.26666667 3.26666667 3.46666667 1.26666667
 1.53333333 1.8        3.46666667 1.         0.66666667 3.8
 0.26666667 2.13333333 4.06666667 0.86666667 0.

[0.13333333 1.4        3.06666667 0.86666667 1.         1.06666667
 1.2        2.6        4.4        4.2        5.         0.86666667
 4.86666667 0.26666667 2.73333333 2.13333333 2.         2.33333333
 2.73333333 4.86666667 2.13333333 4.53333333 2.73333333 2.73333333
 4.2        4.13333333 4.46666667 3.8        2.93333333 0.
 0.66666667 0.66666667 5.         0.46666667 0.66666667 1.2
 1.8        4.66666667 4.         2.46666667 1.86666667 1.33333333
 1.46666667 1.         1.46666667 0.06666667 0.         0.06666667
 2.         0.8        4.8        2.66666667 4.06666667 3.86666667
 1.26666667 3.73333333 3.33333333 0.8        0.6        1.33333333
 3.86666667 4.06666667 4.06666667 0.06666667 0.6        4.66666667
 1.2        1.73333333 3.53333333 2.53333333 3.6        2.8
 2.73333333 0.4        1.46666667 1.06666667 1.93333333 1.8
 3.53333333 2.86666667 0.06666667 2.8        0.2        0.86666667
 1.         2.26666667 5.         3.26666667 1.73333333 0.8
 2.06666667 0.6        1.533333

In [18]:
rfr_pred1 = cross_val_predict(rfr1,tsr_12_input_nomrs,tsr_12_output,cv = 10)
rfr_pred1 = np.round(rfr_pred1)
confusion_matrix(tsr_12_output, rfr_pred1)

array([[209, 107,  14,   6,   0,   1],
       [ 44, 593, 156,  34,   4,   0],
       [  0, 104, 265,  68,   9,   0],
       [  0,  12,  93, 236,  52,   1],
       [  0,   6,  17,  95, 364,  28],
       [  0,   1,   2,  12,  86, 178]], dtype=int64)

## XGBoost

In [19]:
xgbr = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores = cross_val_score(xgbr,tsr_12_input,tsr_12_output,cv = 10, scoring='r2')
print(xgbr_scores)
print("Mean of R^2:", xgbr_scores.mean())
print("Std of R^2:", xgbr_scores.std())

[0.99993885 0.99995166 0.99995083 0.99995068 0.99994955 0.99994612
 0.99995135 0.99994278 0.99994875 0.99993334]
Mean of R^2: 0.9999463923219803
Std of R^2: 5.884427122176203e-06


In [20]:
acc_xgbr = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(tsr_12_input, tsr_12_output, test_size=0.3, random_state=i)
    xgbr.fit(X_train,y_train)
    xgbr_predict = xgbr.predict(X_test)
    print(xgbr_predict)
    xgbr_predict = np.round(xgbr_predict)
    print(xgbr_predict)
    accuracy = (y_test == xgbr_predict).sum() / len(xgbr_predict)
    acc_xgbr.append(accuracy)

print("Accuracy of RF:", acc_xgbr)
print("Mean of Accuracy of RF:", sum(acc_xgbr)/10)

[9.9759996e-01 9.9759996e-01 3.9830739e+00 2.4384370e-03 2.4384370e-03
 9.9759996e-01 2.9878645e+00 1.9927310e+00 2.4384370e-03 9.9759996e-01
 9.9759996e-01 9.9759996e-01 4.9779019e+00 3.9830739e+00 9.9759996e-01
 2.4384370e-03 9.9759996e-01 4.9779019e+00 3.9830739e+00 4.9779019e+00
 2.4384370e-03 9.9759996e-01 4.9779019e+00 9.9759996e-01 2.9878645e+00
 9.9759996e-01 4.9779019e+00 9.9759996e-01 4.9779019e+00 3.9830739e+00
 9.9759996e-01 1.9927310e+00 2.9878645e+00 4.9779019e+00 9.9759996e-01
 9.9759996e-01 4.9779019e+00 2.4384370e-03 9.9759996e-01 9.9759996e-01
 9.9759996e-01 9.9759996e-01 3.9830739e+00 3.9830739e+00 1.9927310e+00
 2.4384370e-03 9.9759996e-01 3.9830739e+00 9.9759996e-01 9.9759996e-01
 4.9779019e+00 9.9759996e-01 2.9878645e+00 9.9759996e-01 2.4384370e-03
 4.9779019e+00 1.9927310e+00 1.9927310e+00 9.9759996e-01 3.9830739e+00
 2.9878645e+00 9.9759996e-01 2.4384370e-03 1.9927310e+00 9.9759996e-01
 9.9759996e-01 2.4384370e-03 2.9878645e+00 9.9759996e-01 9.9759996e-01
 9.975

[2.4378938e-03 4.9779491e+00 4.9779491e+00 9.9759912e-01 2.4378938e-03
 2.9878595e+00 4.9779491e+00 1.9927332e+00 2.9878595e+00 2.4378938e-03
 9.9759912e-01 2.4378938e-03 9.9759912e-01 9.9759912e-01 2.4378938e-03
 9.9759912e-01 9.9759912e-01 2.9878595e+00 9.9759912e-01 2.9878595e+00
 2.9878595e+00 9.9759912e-01 9.9759912e-01 9.9759912e-01 3.9830766e+00
 2.4378938e-03 9.9759912e-01 3.9830766e+00 9.9759912e-01 3.9830766e+00
 3.9830766e+00 2.4378938e-03 9.9759912e-01 3.9830766e+00 9.9759912e-01
 9.9759912e-01 9.9759912e-01 2.4378938e-03 3.9830766e+00 2.4378938e-03
 2.9878595e+00 3.9830766e+00 3.9830766e+00 9.9759912e-01 2.4378938e-03
 4.9779491e+00 4.9779491e+00 1.9927332e+00 3.9830766e+00 2.4378938e-03
 2.9878595e+00 4.9779491e+00 2.4378938e-03 1.9927332e+00 9.9759912e-01
 2.4378938e-03 4.9779491e+00 3.9830766e+00 3.9830766e+00 9.9759912e-01
 2.9878595e+00 9.9759912e-01 9.9759912e-01 2.4378938e-03 9.9759912e-01
 3.9830766e+00 3.9830766e+00 2.4378938e-03 2.4378938e-03 2.9878595e+00
 9.975

[3.9830885e+00 4.9779019e+00 9.9760026e-01 3.9830885e+00 2.4412975e-03
 9.9760026e-01 2.9878623e+00 3.9830885e+00 4.9779019e+00 9.9760026e-01
 2.4412975e-03 3.9830885e+00 4.9779019e+00 9.9760026e-01 9.9760026e-01
 2.4412975e-03 1.9927250e+00 2.9878623e+00 9.9760026e-01 4.9779019e+00
 3.9830885e+00 1.9927250e+00 1.9927250e+00 1.9927250e+00 3.9830885e+00
 9.9760026e-01 1.9927250e+00 9.9760026e-01 9.9760026e-01 3.9830885e+00
 9.9760026e-01 9.9760026e-01 9.9760026e-01 3.9830885e+00 4.9779019e+00
 3.9830885e+00 2.4412975e-03 9.9760026e-01 4.9779019e+00 1.9927250e+00
 3.9830885e+00 1.9927250e+00 1.9927250e+00 9.9760026e-01 2.9878623e+00
 2.9878623e+00 9.9760026e-01 4.9779019e+00 2.9878623e+00 9.9760026e-01
 1.9927250e+00 2.4412975e-03 1.9927250e+00 4.9779019e+00 2.9878623e+00
 4.9779019e+00 2.9878623e+00 1.9927250e+00 1.9927250e+00 4.9779019e+00
 4.9779019e+00 2.9878623e+00 2.4412975e-03 2.4412975e-03 1.9927250e+00
 2.9878623e+00 9.9760026e-01 4.9779019e+00 9.9760026e-01 2.4412975e-03
 9.976

In [21]:
xgbr_pred = cross_val_predict(xgbr,tsr_12_input,tsr_12_output,cv = 10)
xgbr_pred = np.round(xgbr_pred)
confusion_matrix(tsr_12_output, xgbr_pred)

array([[337,   0,   0,   0,   0,   0],
       [  0, 831,   0,   0,   0,   0],
       [  0,   0, 446,   0,   0,   0],
       [  0,   0,   0, 394,   0,   0],
       [  0,   0,   0,   0, 510,   0],
       [  0,   0,   0,   0,   0, 279]], dtype=int64)

In [22]:
xgbr1 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
xgbr_scores1 = cross_val_score(xgbr1,tsr_12_input_nomrs,tsr_12_output,cv = 10, scoring='r2')
print(xgbr_scores1)
print("Mean of R^2:", xgbr_scores1.mean())
print("Std of R^2:", xgbr_scores1.std())

[0.7847378  0.81460552 0.86070665 0.81994466 0.78999239 0.83301935
 0.83633177 0.86456348 0.77757829 0.78390643]
Mean of R^2: 0.8165386343479961
Std of R^2: 0.030388623691578846


In [23]:
acc_xgbr1 = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(tsr_12_input_nomrs, tsr_12_output, test_size=0.3, random_state=i)
    xgbr1.fit(X_train,y_train)
    xgbr1_predict = xgbr1.predict(X_test)
    print(xgbr1_predict)
    xgbr1_predict = np.round(xgbr1_predict)
    print(xgbr1_predict)
    accuracy = (y_test == xgbr1_predict).sum() / len(xgbr1_predict)
    acc_xgbr1.append(accuracy)

print("Accuracy of RF:", acc_xgbr1)
print("Mean of Accuracy of RF:", sum(acc_xgbr1)/10)

[0.06666667 0.2        3.73333333 0.13333333 0.06666667 1.
 2.73333333 2.73333333 0.26666667 1.         1.6        1.2
 4.93333333 3.93333333 1.6        0.13333333 1.46666667 4.73333333
 4.13333333 4.86666667 1.         1.53333333 4.26666667 1.
 2.66666667 1.06666667 3.13333333 1.         4.73333333 3.33333333
 0.6        1.33333333 2.66666667 5.         1.06666667 1.
 4.06666667 0.33333333 1.2        4.         1.2        1.
 3.33333333 3.46666667 2.53333333 0.06666667 1.         3.86666667
 1.         1.2        4.26666667 1.         3.73333333 1.2
 0.06666667 4.4        1.4        1.73333333 1.6        2.93333333
 2.4        0.86666667 1.06666667 2.73333333 2.8        0.6
 0.06666667 2.66666667 1.6        1.4        1.         0.26666667
 3.86666667 1.         3.53333333 1.8        1.06666667 2.86666667
 4.8        1.93333333 2.8        0.86666667 1.06666667 0.8
 0.93333333 2.2        4.26666667 1.4        5.         0.8
 0.86666667 2.6        4.66666667 4.2        3.6        1.2
 3

[0.93333333 0.8        4.93333333 1.46666667 4.13333333 1.
 1.06666667 0.06666667 2.06666667 1.13333333 4.93333333 1.
 2.53333333 2.73333333 1.         0.53333333 1.66666667 3.
 3.6        0.         2.06666667 4.33333333 3.8        1.06666667
 0.93333333 1.26666667 3.2        0.93333333 1.4        3.8
 1.6        4.66666667 1.         3.06666667 4.2        3.93333333
 0.73333333 1.         3.73333333 1.73333333 1.33333333 2.53333333
 4.4        1.8        1.13333333 1.33333333 4.         3.46666667
 0.06666667 3.86666667 1.46666667 0.2        0.13333333 1.26666667
 3.93333333 1.         0.93333333 0.8        2.4        3.46666667
 0.86666667 1.86666667 2.53333333 1.73333333 1.53333333 2.93333333
 1.46666667 4.2        1.         0.2        5.         0.93333333
 1.         0.06666667 4.86666667 0.46666667 4.         1.66666667
 1.66666667 3.53333333 0.2        0.26666667 1.         1.66666667
 3.8        0.8        2.13333333 2.13333333 3.73333333 4.46666667
 4.2        2.6        0.8

[4.66666667 0.73333333 1.73333333 3.06666667 4.66666667 4.53333333
 1.06666667 1.53333333 0.06666667 1.         1.73333333 4.86666667
 0.4        2.86666667 4.06666667 4.13333333 3.8        0.46666667
 1.8        1.26666667 0.93333333 1.2        2.86666667 1.66666667
 1.53333333 5.         3.         2.         0.86666667 2.93333333
 1.46666667 0.4        4.46666667 4.         2.06666667 4.
 2.26666667 1.66666667 3.6        0.86666667 3.26666667 2.8
 4.8        2.53333333 3.6        2.33333333 4.33333333 1.
 1.86666667 0.2        0.33333333 1.4        3.13333333 3.2
 4.06666667 3.93333333 4.8        1.         1.13333333 1.53333333
 3.         1.8        0.86666667 1.8        2.73333333 1.53333333
 0.         2.93333333 4.93333333 2.46666667 1.33333333 0.8
 4.         1.53333333 4.8        3.93333333 0.13333333 2.66666667
 5.         2.2        0.86666667 0.26666667 4.6        0.86666667
 0.33333333 0.13333333 2.73333333 4.33333333 0.93333333 1.53333333
 1.66666667 1.46666667 4.7333333

[0.8        1.         4.13333333 2.         1.         3.06666667
 4.66666667 1.53333333 2.86666667 3.33333333 1.13333333 0.13333333
 0.73333333 1.06666667 0.4        1.         2.53333333 1.73333333
 1.         2.53333333 3.13333333 1.8        0.93333333 1.53333333
 2.93333333 0.06666667 0.73333333 4.6        1.         4.33333333
 3.86666667 0.06666667 0.86666667 3.93333333 1.8        1.
 0.93333333 0.13333333 3.6        1.2        2.66666667 3.86666667
 3.86666667 1.         0.26666667 4.66666667 4.26666667 2.
 2.8        0.13333333 3.06666667 4.4        0.2        1.86666667
 1.26666667 0.33333333 4.8        3.33333333 3.93333333 0.93333333
 3.13333333 0.93333333 1.         0.2        1.33333333 3.33333333
 4.13333333 0.4        0.         3.93333333 1.06666667 1.6
 2.86666667 3.6        3.86666667 2.86666667 3.93333333 4.6
 1.         0.13333333 1.06666667 4.13333333 1.         1.
 3.33333333 4.13333333 2.86666667 1.         1.8        3.93333333
 2.         3.13333333 4.6       

[3.46666667 2.93333333 4.26666667 4.06666667 2.53333333 2.46666667
 4.93333333 1.66666667 0.8        2.2        0.         2.53333333
 0.         3.86666667 0.         0.         1.46666667 2.53333333
 1.26666667 4.2        0.8        0.2        1.         0.2
 1.46666667 1.         0.         0.06666667 1.33333333 2.86666667
 4.33333333 1.8        2.8        0.8        2.6        3.
 1.06666667 4.13333333 3.06666667 4.33333333 3.06666667 2.93333333
 0.8        2.86666667 1.13333333 0.8        4.66666667 0.93333333
 1.73333333 2.6        1.6        3.         0.93333333 0.93333333
 1.86666667 1.         1.46666667 3.8        2.6        0.86666667
 1.4        4.73333333 2.66666667 0.53333333 2.86666667 1.8
 0.         1.06666667 1.6        3.         5.         3.93333333
 4.86666667 0.         5.         2.93333333 4.46666667 0.93333333
 1.73333333 2.6        1.8        1.06666667 0.06666667 1.06666667
 4.         5.         0.66666667 1.66666667 2.93333333 0.
 3.73333333 0.06666667 0.

[1.6        0.93333333 0.86666667 2.13333333 1.6        3.86666667
 0.8        2.         3.4        4.         0.93333333 3.93333333
 2.13333333 1.53333333 4.2        2.2        2.2        1.06666667
 1.6        2.8        2.6        0.86666667 1.33333333 4.93333333
 0.86666667 3.13333333 4.         0.46666667 2.6        4.13333333
 1.         1.6        4.8        4.53333333 4.53333333 4.6
 4.26666667 3.4        3.86666667 0.93333333 3.13333333 1.6
 1.66666667 0.4        2.8        3.93333333 4.53333333 0.33333333
 0.86666667 1.33333333 1.33333333 0.13333333 1.93333333 1.86666667
 2.46666667 0.93333333 1.         0.06666667 2.73333333 1.93333333
 3.6        1.73333333 4.06666667 0.86666667 0.66666667 0.86666667
 1.6        3.4        0.93333333 0.06666667 1.6        4.33333333
 4.26666667 1.66666667 2.2        0.93333333 3.66666667 0.13333333
 1.73333333 1.26666667 3.93333333 2.86666667 1.06666667 4.
 2.8        5.         4.66666667 1.46666667 1.8        1.6
 1.33333333 2.         2

[1.73333333 1.46666667 1.53333333 4.66666667 0.         1.46666667
 2.93333333 1.73333333 0.13333333 4.73333333 3.8        1.46666667
 0.2        2.         3.33333333 2.33333333 2.6        0.93333333
 0.86666667 4.66666667 2.33333333 1.46666667 1.6        3.86666667
 3.53333333 3.73333333 2.26666667 2.66666667 1.2        3.66666667
 4.2        1.8        1.66666667 0.         2.4        0.93333333
 4.06666667 0.6        4.86666667 1.2        0.4        4.26666667
 0.73333333 3.46666667 1.86666667 4.86666667 0.93333333 4.86666667
 1.73333333 0.06666667 3.8        1.06666667 1.53333333 3.2
 1.73333333 2.86666667 3.6        2.46666667 1.         4.26666667
 2.8        2.6        4.53333333 2.06666667 5.         1.
 1.53333333 3.26666667 1.4        3.73333333 1.06666667 1.73333333
 0.93333333 2.8        1.06666667 1.06666667 4.53333333 3.8
 4.6        1.8        0.         0.73333333 2.66666667 4.06666667
 0.86666667 3.2        1.6        1.73333333 1.06666667 3.4
 4.8        1.26666667 3

[4.26666667 4.8        1.         2.53333333 0.33333333 0.86666667
 2.86666667 3.73333333 4.8        1.46666667 0.13333333 4.06666667
 4.93333333 0.8        1.66666667 0.06666667 1.6        3.93333333
 0.93333333 4.13333333 3.86666667 1.66666667 1.86666667 1.46666667
 3.8        0.86666667 2.2        1.         0.93333333 4.13333333
 1.06666667 2.73333333 2.         3.2        4.33333333 3.86666667
 0.4        1.66666667 4.8        1.46666667 4.33333333 1.6
 1.26666667 1.6        3.93333333 2.6        0.8        4.4
 2.66666667 1.66666667 1.8        0.4        2.8        2.93333333
 3.06666667 4.86666667 2.86666667 2.6        1.6        4.86666667
 4.86666667 2.8        0.2        0.73333333 1.6        2.86666667
 1.53333333 5.         2.13333333 0.13333333 1.13333333 2.86666667
 0.93333333 3.06666667 2.86666667 0.         0.93333333 1.2
 0.33333333 2.         0.86666667 2.06666667 1.86666667 1.53333333
 0.46666667 4.8        1.53333333 0.53333333 3.         0.86666667
 3.86666667 3.86

[1.         4.53333333 2.33333333 0.93333333 1.06666667 3.73333333
 4.26666667 0.86666667 3.13333333 2.53333333 1.73333333 4.33333333
 3.73333333 2.8        4.13333333 2.8        2.         1.
 2.93333333 4.66666667 1.6        1.13333333 3.53333333 1.
 3.66666667 2.26666667 0.93333333 1.73333333 3.2        3.53333333
 1.73333333 1.         0.93333333 2.53333333 1.         1.
 3.13333333 4.6        4.4        0.93333333 1.66666667 2.2
 2.86666667 3.73333333 0.86666667 1.13333333 1.73333333 2.8
 2.06666667 3.93333333 1.2        1.8        2.8        2.46666667
 2.53333333 3.26666667 4.2        1.4        4.06666667 4.86666667
 1.46666667 2.06666667 2.86666667 4.13333333 4.06666667 1.6
 4.8        3.93333333 3.86666667 0.73333333 2.46666667 4.13333333
 1.13333333 3.         2.8        3.2        1.33333333 3.93333333
 4.         0.26666667 4.26666667 3.26666667 3.46666667 1.26666667
 1.53333333 1.8        3.46666667 1.         0.66666667 3.8
 0.26666667 2.13333333 4.06666667 0.86666667 0.

[0.13333333 1.4        3.06666667 0.86666667 1.         1.06666667
 1.2        2.6        4.4        4.2        5.         0.86666667
 4.86666667 0.26666667 2.73333333 2.13333333 2.         2.33333333
 2.73333333 4.86666667 2.13333333 4.53333333 2.73333333 2.73333333
 4.2        4.13333333 4.46666667 3.8        2.93333333 0.
 0.66666667 0.66666667 5.         0.46666667 0.66666667 1.2
 1.8        4.66666667 4.         2.46666667 1.86666667 1.33333333
 1.46666667 1.         1.46666667 0.06666667 0.         0.06666667
 2.         0.8        4.8        2.66666667 4.06666667 3.86666667
 1.26666667 3.73333333 3.33333333 0.8        0.6        1.33333333
 3.86666667 4.06666667 4.06666667 0.06666667 0.6        4.66666667
 1.2        1.73333333 3.53333333 2.53333333 3.6        2.8
 2.73333333 0.4        1.46666667 1.06666667 1.93333333 1.8
 3.53333333 2.86666667 0.06666667 2.8        0.2        0.86666667
 1.         2.26666667 5.         3.26666667 1.73333333 0.8
 2.06666667 0.6        1.533333

In [24]:
xgbr_pred1 = cross_val_predict(xgbr1,tsr_12_input_nomrs,tsr_12_output,cv = 10)
xgbr_pred1 = np.round(xgbr_pred1)
confusion_matrix(tsr_12_output, xgbr_pred1)

array([[209, 107,  14,   6,   0,   1],
       [ 44, 593, 156,  34,   4,   0],
       [  0, 104, 265,  68,   9,   0],
       [  0,  12,  93, 236,  52,   1],
       [  0,   6,  17,  95, 364,  28],
       [  0,   1,   2,  12,  86, 178]], dtype=int64)

# 2 classes

In [25]:
tsr_12_output[(tsr_12_output == 0)|(tsr_12_output == 1)|(tsr_12_output == 2)] = 0
tsr_12_output[(tsr_12_output == 3)|(tsr_12_output == 4)|(tsr_12_output == 5)] = 1

## SVM

In [26]:
svr2 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores2 = cross_val_score(svr2,tsr_12_input,tsr_12_output,cv = 10, scoring='r2')
print(svr_scores2)
print("Mean of R^2:", svr_scores2.mean())
print("Std of R^2:", svr_scores2.std())

[0.73158965 0.80185008 0.83005761 0.79631666 0.82429108 0.83559804
 0.81749126 0.752905   0.74944993 0.75265321]
Mean of R^2: 0.7892202514995923
Std of R^2: 0.03690065588657889


In [27]:
acc_svr2 = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(tsr_12_input, tsr_12_output, test_size=0.3, random_state=i)
    svr2.fit(X_train,y_train)
    svr2_predict = svr2.predict(X_test)
    #print(svr2_predict)
    svr2_predict = np.round(svr2_predict)
    #print(svr2_predict)
    accuracy = (y_test == svr2_predict).sum() / len(svr2_predict)
    acc_svr2.append(accuracy)

print("Accuracy of RF:", acc_svr2)
print("Mean of Accuracy of RF:", sum(acc_svr2)/10)

Accuracy of RF: [0.9595238095238096, 0.9833333333333333, 0.9738095238095238, 0.9607142857142857, 0.9797619047619047, 0.9714285714285714, 0.9285714285714286, 0.9773809523809524, 0.955952380952381, 0.95]
Mean of Accuracy of RF: 0.9640476190476189


In [28]:
svr_pred2 = cross_val_predict(svr2,tsr_12_input,tsr_12_output,cv = 10)
svr_pred2 = np.round(svr_pred2)
confusion_matrix(tsr_12_output, svr_pred2)

array([[   0,    0,    0],
       [   3, 1563,   48],
       [   0,   63, 1120]], dtype=int64)

In [29]:
svr_pred2[svr_pred2 == -1] = 0
confusion_matrix(tsr_12_output, svr_pred2)

array([[1566,   48],
       [  63, 1120]], dtype=int64)

In [30]:
svr3 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores3 = cross_val_score(svr3,tsr_12_input_nomrs,tsr_12_output,cv = 10, scoring='r2')
print(svr_scores3)
print("Mean of R^2:", svr_scores3.mean())
print("Std of R^2:", svr_scores3.std())

[0.59355326 0.64711454 0.73387118 0.68933979 0.70835016 0.7730554
 0.69816056 0.6910978  0.63857463 0.58865123]
Mean of R^2: 0.6761768549082917
Std of R^2: 0.056012792963407616


In [31]:
acc_svr3 = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(tsr_12_input_nomrs, tsr_12_output, test_size=0.3, random_state=i)
    svr3.fit(X_train,y_train)
    svr3_predict = svr3.predict(X_test)
    #print(svr3_predict)
    svr3_predict = np.round(svr3_predict)
    #print(svr3_predict)
    accuracy = (y_test == svr3_predict).sum() / len(svr3_predict)
    acc_svr3.append(accuracy)

print("Accuracy of RF:", acc_svr3)
print("Mean of Accuracy of RF:", sum(acc_svr3)/10)

Accuracy of RF: [0.9, 0.9035714285714286, 0.8976190476190476, 0.9142857142857143, 0.9166666666666666, 0.9238095238095239, 0.8845238095238095, 0.8988095238095238, 0.8952380952380953, 0.9083333333333333]
Mean of Accuracy of RF: 0.9042857142857142


In [32]:
svr_pred3 = cross_val_predict(svr3,tsr_12_input_nomrs,tsr_12_output,cv = 10)
svr_pred3 = np.round(svr_pred3)
confusion_matrix(tsr_12_output, svr_pred3)

array([[1479,  135,    0],
       [ 129, 1053,    1],
       [   0,    0,    0]], dtype=int64)

In [33]:
svr_pred3[svr_pred3 == 2] = 1

confusion_matrix(tsr_12_output, svr_pred3)

array([[1479,  135],
       [ 129, 1054]], dtype=int64)

## RF

In [34]:
rfr2 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores2 = cross_val_score(rfr2,tsr_12_input,tsr_12_output,cv = 10, scoring='r2')
print(rfr_scores2)
print("Mean of R^2:", rfr_scores2.mean())
print("Std of R^2:", rfr_scores2.std())

[1.         0.99980424 1.         0.99992919 1.         1.
 1.         1.         1.         1.        ]
Mean of R^2: 0.9999733432645282
Std of R^2: 6.019117035794216e-05


In [35]:
acc_rfr2 = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(tsr_12_input, tsr_12_output, test_size=0.3, random_state=i)
    rfr2.fit(X_train,y_train)
    rfr2_predict = rfr2.predict(X_test)
    #print(rfr2_predict)
    rfr2_predict = np.round(rfr2_predict)
    #print(rfr2_predict)
    accuracy = (y_test == rfr2_predict).sum() / len(rfr2_predict)
    acc_rfr2.append(accuracy)

print("Accuracy of RF:", acc_rfr2)
print("Mean of Accuracy of RF:", sum(acc_rfr2)/10)

Accuracy of RF: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Mean of Accuracy of RF: 1.0


In [36]:
rf_pred2 = cross_val_predict(rfr2,tsr_12_input,tsr_12_output,cv = 10)
rf_pred2 = np.round(rf_pred2)
confusion_matrix(tsr_12_output, rf_pred2)

array([[1614,    0],
       [   0, 1183]], dtype=int64)

In [37]:
rfr3 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores3 = cross_val_score(rfr3,tsr_12_input_nomrs,tsr_12_output,cv = 10, scoring='r2')
print(rfr_scores3)
print("Mean of R^2:", rfr_scores3.mean())
print("Std of R^2:", rfr_scores3.std())

[0.67436952 0.70433759 0.7949208  0.74424277 0.75273339 0.79387526
 0.7098545  0.73176183 0.6800626  0.60852235]
Mean of R^2: 0.7194680609420423
Std of R^2: 0.054146120458681925


In [38]:
acc_rfr3 = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(tsr_12_input_nomrs, tsr_12_output, test_size=0.3, random_state=i)
    rfr3.fit(X_train,y_train)
    rfr3_predict = rfr3.predict(X_test)
    #print(rfr3_predict)
    rfr3_predict = np.round(rfr3_predict)
    #print(rfr3_predict)
    accuracy = (y_test == rfr3_predict).sum() / len(rfr3_predict)
    acc_rfr3.append(accuracy)

print("Accuracy of RF:", acc_rfr3)
print("Mean of Accuracy of RF:", sum(acc_rfr3)/10)

Accuracy of RF: [0.9166666666666666, 0.9071428571428571, 0.9166666666666666, 0.9119047619047619, 0.9226190476190477, 0.9142857142857143, 0.9035714285714286, 0.9023809523809524, 0.9071428571428571, 0.9238095238095239]
Mean of Accuracy of RF: 0.9126190476190477


In [39]:
rfr_pred3 = cross_val_predict(rfr3,tsr_12_input_nomrs,tsr_12_output,cv = 10)
rfr_pred3 = np.round(rfr_pred3)
confusion_matrix(tsr_12_output, rfr_pred3)

array([[1462,  152],
       [  93, 1090]], dtype=int64)

## XGBoost

In [40]:
xgbr2 = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores2 = cross_val_score(xgbr2,tsr_12_input,tsr_12_output,cv = 10, scoring='r2')
print(xgbr_scores2)
print("Mean of R^2:", xgbr_scores2.mean())
print("Std of R^2:", xgbr_scores2.std())

[0.99997706 0.9999766  0.99997648 0.99997461 0.99997654 0.999977
 0.99997628 0.99997644 0.99997609 0.99997706]
Mean of R^2: 0.9999764165186296
Std of R^2: 6.781968422816103e-07


In [41]:
acc_xgbr2 = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(tsr_12_input, tsr_12_output, test_size=0.3, random_state=i)
    xgbr2.fit(X_train,y_train)
    xgbr2_predict = xgbr2.predict(X_test)
    #print(xgbr2_predict)
    xgbr2_predict = np.round(xgbr2_predict)
    #print(xgbr2_predict)
    accuracy = (y_test == xgbr2_predict).sum() / len(xgbr2_predict)
    acc_xgbr2.append(accuracy)

print("Accuracy of RF:", acc_xgbr2)
print("Mean of Accuracy of RF:", sum(acc_xgbr2)/10)

Accuracy of RF: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Mean of Accuracy of RF: 1.0


In [42]:
xgbr_pred2 = cross_val_predict(xgbr2,tsr_12_input,tsr_12_output,cv = 10)
xgbr_pred2 = np.round(xgbr_pred2)
confusion_matrix(tsr_12_output, xgbr_pred2)

array([[1614,    0],
       [   0, 1183]], dtype=int64)

In [43]:
xgbr3 = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores3 = cross_val_score(xgbr3,tsr_12_input_nomrs,tsr_12_output,cv = 10, scoring='r2')
print(xgbr_scores3)
print("Mean of R^2:", xgbr_scores3.mean())
print("Std of R^2:", xgbr_scores3.std())

[0.69106629 0.72264037 0.82978793 0.75276784 0.76755001 0.79607017
 0.68396137 0.74830537 0.67251316 0.6428272 ]
Mean of R^2: 0.7307489717278237
Std of R^2: 0.055833807794982114


In [44]:
acc_xgbr3 = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(tsr_12_input_nomrs, tsr_12_output, test_size=0.3, random_state=i)
    xgbr3.fit(X_train,y_train)
    xgbr3_predict = xgbr3.predict(X_test)
    #print(xgbr3_predict)
    xgbr3_predict = np.round(xgbr3_predict)
    #print(xgbr3_predict)
    accuracy = (y_test == xgbr3_predict).sum() / len(xgbr3_predict)
    acc_xgbr3.append(accuracy)

print("Accuracy of RF:", acc_xgbr3)
print("Mean of Accuracy of RF:", sum(acc_xgbr3)/10)

Accuracy of RF: [0.9166666666666666, 0.9083333333333333, 0.9142857142857143, 0.9226190476190477, 0.919047619047619, 0.9202380952380952, 0.8964285714285715, 0.8940476190476191, 0.9011904761904762, 0.9226190476190477]
Mean of Accuracy of RF: 0.911547619047619


In [45]:
xgbr_pred3 = cross_val_predict(xgbr3,tsr_12_input_nomrs,tsr_12_output,cv = 10)
xgbr_pred3 = np.round(xgbr_pred3)
confusion_matrix(tsr_12_output, xgbr_pred3)

array([[1472,  142],
       [  95, 1088]], dtype=int64)

# Summary

## Mean & Std

In [46]:
svr_mean = np.array([svr_scores.mean(), svr_scores.std(), svr_scores[0], svr_scores[1], svr_scores[2], svr_scores[3],
                     svr_scores[4], svr_scores[5], svr_scores[6], svr_scores[7], svr_scores[8], svr_scores[9]])
rfr_mean = np.array([rfr_scores.mean(), rfr_scores.std(), rfr_scores[0], rfr_scores[1], rfr_scores[2], rfr_scores[3],rfr_scores[4], 
                    rfr_scores[5], rfr_scores[6], rfr_scores[7], rfr_scores[8], rfr_scores[9]])
xgbr_mean = np.array([xgbr_scores.mean(), xgbr_scores.std(), xgbr_scores[0], xgbr_scores[1], xgbr_scores[2], xgbr_scores[3],
                     xgbr_scores[4], xgbr_scores[5], xgbr_scores[6], xgbr_scores[7], xgbr_scores[8], xgbr_scores[9]])
svr_mean2 = np.array([svr_scores2.mean(), svr_scores2.std(), svr_scores2[0], svr_scores2[1], svr_scores2[2], svr_scores2[3],
                     svr_scores2[4], svr_scores2[5], svr_scores2[6], svr_scores2[7], svr_scores2[8], svr_scores2[9]])
rfr_mean2 = np.array([rfr_scores2.mean(), rfr_scores2.std(), rfr_scores2[0], rfr_scores2[1], rfr_scores2[2], rfr_scores2[3],rfr_scores2[4], 
                    rfr_scores2[5], rfr_scores2[6], rfr_scores2[7], rfr_scores2[8], rfr_scores2[9]])
xgbr_mean2 = np.array([xgbr_scores2.mean(), xgbr_scores2.std(), xgbr_scores2[0], xgbr_scores2[1], xgbr_scores2[2], xgbr_scores2[3],
                     xgbr_scores2[4], xgbr_scores2[5], xgbr_scores2[6], xgbr_scores2[7], xgbr_scores2[8], xgbr_scores2[9]])

In [47]:
tsr_12_mean = pd.DataFrame([svr_mean, rfr_mean, xgbr_mean, svr_mean2, rfr_mean2, xgbr_mean2]).T
tsr_12_mean.index = ["Mean", "Std", "R^2_1", "R^2_2", "R^2_3", "R^2_4", "R^2_5", "R^2_6", "R^2_7", "R^2_8", "R^2_9", "R^2_10"]
tsr_12_mean.columns = ["svr", 'rfr', 'xgbr', 'svr2', 'rfr2', 'xgbr2']

In [48]:
csv_save = os.path.join(".", "tsr_12_mean_regression.csv")
tsr_12_mean.to_csv(csv_save, index = True)